# **4: Training Corpus Creation**
**1. Creation of Training Corpus for SpaCy NER Training**<br>
`Input: refined_text.csv`<br>
`Output: training_corpus.csv`

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
corpus=pd.read_csv('/content/drive/MyDrive/Final (Completed)/refined_text2.csv')#,index_col=0)
pfs=pd.read_csv('/content/drive/MyDrive/Final (Completed)/process_flow_sentences_2.csv')#,index_col=0)

Get entities and labels from `process_flow_sentences.csv`

In [ ]:
corpus

In [ ]:
#Create Unique Entity-Label Pairs
#Get ent_1, lab_1 and ent_2, lab_2 as tuples('entity','label') in a list
#Remove Duplicates
entities=[] #list of unique entities
labels=[] #list of unique labels
el_dict={} #Dictionary of Entities and Labels
el= [] #list of entities and labels
#Getting List of tuples(entity,labels)
for i in range(0,len(pfs)):
    el.append((pfs['ent_1'][i],pfs['lab_1'][i]))
    el.append((pfs['ent_2'][i],pfs['lab_2'][i]))
el= [*set(el)] 
#Getting dictionary{entity:label} and list of Unique Labels
for t in el:
    el_dict[t[0]]=t[1]
    labels.append(t[1])
    entities.append(t[0])
labels=[*set(labels)]
entities=[*set(entities)]
entities.sort(key=len,reverse=True)

In [ ]:
el

Below we save to disk, the **unique labes** as `labels.csv` and **Entity-Label pairs** as `ent_labels.csv`

In [ ]:
l=pd.DataFrame(labels)
l.to_csv('/content/drive/MyDrive/Final (Completed)/labels.csv',index=False)
#(pd.DataFrame(el)).to_csv('/content/drive/MyDrive/Final (Completed)/ent_labels.csv',index=False)
l.rename(columns={0:"labels"},inplace=True)
l

,labels
0,DOCUMENT
1,PROCESS
2,PERSON
3,PORTAL
4,MOBILE_APP
5,CSC
6,PERMISSION
7,ORG


Now we have the following:<br>
**`entities `**`: List of all unique entities.`<br>
**`labels `**`: List of all unique labels.`<br>
**`el `**`: List of unique Tuples(entity, label)`<br>
**`el_dict :`**`Dictionary Entity:Label`<br>
* Use entity from `entities` to find its start and end index in a sentence

Find entities in the sentences:<br>
* Find it's start and end index
* Create the format: `t_data= [(sentence,{'entities':[(start_ind,end_ind,label),...]}),...]`

In [ ]:
#Find entities in the sentences and annotate them with relevant labels
print("Finding entities in the sentences and annotating them with relevant labels")
import re
t=[] #List of Annotated sentences
for s in corpus['sentences']:
    index=[] #Stores start and end indices of entities in a sentence along with their labels
    for e in entities:
        #get start and end index of tok.text
        for i in re.finditer(e, s): #Finds indices of entity e in sentence s
            index.append((i.start(),i.end(),el_dict[e]))
    index=sorted(index)
    t.append((s,index))
print("Sample:\n",t[505])

Finding entities in the sentences and annotating them with relevant labels
Sample:
 ('PMJAY has made healthcare more accessible for people by providing them with free Empanelled Hospitalization through a network of General Doctor.', [(0, 5, 'ORG'), (81, 100, 'ORG'), (92, 107, 'PROCESS'), (129, 143, 'PERSON'), (137, 143, 'PERSON')])


In [ ]:
#Remove Sentences with no entities
print("Removing sentences which have no entities in it.")
delete_ind=[]
count=1
for i in range(0,len(t)):
    if(t[i][1]==[]):
        delete_ind.append(i)
delete_ind.sort(reverse=True)
len1= len(t)
for ind in delete_ind:
    t.pop(ind)
len2= len(t)
print("Removed ",len1-len2," sentences which have no entities.")

Removing sentences which have no entities in it.
Removed  53  sentences which have no entities.


### Removing Overlapping Entities<br>
Steps :-<br>
1. Identify entities with same start_index OR same end_index
2. Find difference in indices
3. Remove the entity with lesser difference value<br>


In [ ]:
#Removing Overlapping Entities
for k in range(0,len(t)):
    checked_ind=[] #indices that has already been checked
    delete_ind=[] #Indices of those entities that needs to be removed
    for i in range(0,len(t[k][1])-1):
        if(i in checked_ind):
            #print("Found already checked index at",i,"! Go to Next Index.")
            continue
        else:
            for j in range(i+1,len(t[k][1])):
                s1=t[k][1][i][0] #start index of 1st entity
                e1=t[k][1][i][1] #end index of 1st entity 
                s2=t[k][1][j][0] #start index of 2nd entity
                e2=t[k][1][j][1] #start index of 2nd entity
                if(e1 >= s2):#Step 1: Detection of Overlapping Entities
                    checked_ind.append(j)
                    #Step 2: Getting indices that need to be deleted
                    e1_diff= e1-s1
                    e2_diff= e2-s2
                    if(e1_diff > e2_diff):
                        delete_ind.append(j)
                    else:
                        delete_ind.append(i)

    if(delete_ind != []):#Step 3: Deletion
        #Popping an item will reset the index of the list
        #The indices in delete_ind shouldn't be an index of any other annotation or be out of bound.
        #Sort the delete_ind in reverse order so that popping doesn't cause problems
        delete_ind.sort(reverse=True)
        for ind in range(0,len(delete_ind)): #Pop
            #print(t[k][1])
            t[k][1].pop(delete_ind[ind])

In [ ]:
training_corpus=[]
for r in t:
    training_corpus.append((r[0],{'entities':r[1]}))
training_corpus


In [ ]:
training_corpus[0][1]

{'entities': [(17, 34, 'PERSON'),
  (67, 72, 'ORG'),
  (131, 136, 'ORG'),
  (138, 143, 'ORG')]}

In [ ]:
#Saving the training corpus
tc=pd.DataFrame(training_corpus)
tc.rename(columns={0:"sentences",1:"annotations"},inplace=True)
tc.to_csv('/content/drive/MyDrive/Final (Completed)/training_corpus.csv',index=False)

In [ ]:
tc

,sentences,annotations
0,Microsoft Word - PMJAY Beneficiary Empowerment...,"{'entities': [(17, 34, 'PERSON'), (67, 72, 'OR..."
1,PMJAY is an ambitious government scheme which ...,"{'entities': [(0, 5, 'ORG'), (153, 172, 'ORG')..."
2,The NHA is committed to ensuring that PMJAY Be...,"{'entities': [(4, 7, 'ORG'), (38, 55, 'PERSON')]}"
3,"With this spirit , NHA is sharing the PMJAY Be...","{'entities': [(19, 22, 'ORG'), (38, 55, 'PERSO..."
4,We sincerely hope that the SHA and other stake...,"{'entities': [(27, 30, 'ORG'), (71, 76, 'ORG')..."
...,...,...
1285,PMJAY helps PMJAY Beneficiary to access qualit...,"{'entities': [(0, 5, 'ORG'), (12, 29, 'PERSON'..."
1286,PMJAY empowers PMJAY Beneficiary to make infor...,"{'entities': [(0, 5, 'ORG'), (15, 32, 'PERSON'..."
1287,PMJAY ensures that all Empanelled Hospital adh...,"{'entities': [(0, 5, 'ORG'), (23, 42, 'ORG'), ..."
1288,PMJAY provides a sense of security to PMJAY Be...,"{'entities': [(0, 5, 'ORG'), (38, 55, 'PERSON'..."


# End Of File